In [21]:
# This code is adapted from [How I Created an AI Version of Myself] by Keith McNulty
# Original repository: https://github.com/keithmcnulty/rag_gemma_regression_book/blob/main/rag_gemma_experiment_open.ipynb
# Modifications by Aleksand Botvin
# I have made the following modifications:
#  - Read PDF files
#  - Using serverless LLM
#  - Creating a Telegram bot as an interface

In [23]:
# Установка необходимых пакетов:
# pip install openai

# Импортируем библиотеку для работы с ChromaDB
import chromadb
from chromadb.utils import embedding_functions
from chromadb.utils.batch_utils import create_batches

# Импортируем клиент OpenAI для работы с API OpenRouter
from openai import OpenAI

In [31]:
# Указываем путь к локальной базе данных ChromaDB
CHROMA_DATA_PATH = "chroma_data_bayesium/"

# Используем предобученную модель для получения эмбеддингов
EMBED_MODEL = "all-MiniLM-L6-v2"

# Название коллекции, в которой хранятся документы
COLLECTION_NAME = "bayesium"

# Инициализируем клиента ChromaDB для работы с персистентным хранилищем
chromadb_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

# Переинициализируем функцию эмбеддинга с использованием выбранной модели
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

# Получаем уже созданную коллекцию с книгами по байесовской статистике
collection = chromadb_client.get_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func  
)

In [33]:
# Создаём клиента OpenAI для доступа к API OpenRouter 
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="<ВАШ API KEY>", 
)

In [37]:
def ask_question(question: str, n_docs: int = 3) -> str:
    """
    Функция для обработки вопроса.
    
    Аргументы:
      question (str): Текст вопроса.
      n_docs (int): Количество документов, извлекаемых из базы для формирования ответа.
      
    Возвращает:
      str: Сгенерированный ответ LLM.
    """
    
    # Получаем коллекцию из базы ChromaDB
    collection = chromadb_client.get_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_func  
    )
    
    # Выполняем поиск по схожести: получаем n_docs наиболее релевантных документов
    results = collection.query(
        query_texts=[question],
        n_results=n_docs
    )
    
    # Формируем контекст с текстом документа и соответствующими метаданными (глава, источник)
    context_with_metadata = ""
    referenced_chapters = set()  # Множество для хранения уникальных названий глав
    referenced_sources = set()   # Множество для хранения уникальных источников

    # Проходим по найденным документам
    for i, document in enumerate(results['documents'][0]):
        # Получаем метаданные для текущего документа
        metadata = results['metadatas'][0][i]  
        chapter = metadata.get('chapter', 'Unknown Chapter')  
        referenced_chapters.add(chapter) 
        source = metadata.get('source') 
        referenced_sources.add(source)
        
        # Формируем строку с информацией о документе
        context_with_metadata += f"Document {i+1} (Chapter: {chapter}):\n"
        context_with_metadata += f"{document}\n"
        context_with_metadata += "-" * 50 + "\n"

    # Объединяем названия глав и источники в строку для дальнейшего упоминания в ответе
    chapters_list = ", ".join(referenced_chapters)
    sources = ", ".join(referenced_sources)

    # Формируем окончательный запрос для LLM, включающий вопрос и контекст
    prompt = f"""
    You are an expert on statistics and its applications to analytics.
    Here is a question: {question}
    Answer it strictly based on the following information:
    {context_with_metadata}
    When providing the answer, reference the chapters used by stating "This information is from {chapters_list} by {sources}".
    If the context does not provide enough information, say "I cannot answer this question based on the provided context."
    """

    # Оборачиваем запрос в формате сообщений для модели
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Отправляем запрос на генерацию ответа LLM с помощью OpenRouter
    completion = client.chat.completions.create(
        model="meta-llama/llama-3.1-70b-instruct:free", 
        messages=messages,
        max_tokens=1000,
        temperature=0.25 
    )

    # Возвращаем сгенерированный ответ
    return completion.choices[0].message.content


In [41]:
question = "Give an example of regression using the Ulam function in R."

answer = ask_question(question)
print(answer)

This information is from Chapter 9: Markov Chain Monte Carlo, Chapter 15: Missing Data and Other Opportunities by McElreath R.

To provide an example of regression using the ulam function in R, we can use the terrain ruggedness example from Chapter 9. First, we need to load the data and transform the predictors:

```r
library(rethinking)
data(rugged)
d <- rugged
d$log_gdp <- log(d$rgdppc_2000)
dd <- d[ complete.cases(d$rgdppc_2000) , ]
dd$log_gdp_std <- dd$log_gdp / mean(dd$log_gdp)
dd$rugged_std <- dd$rugged / max(dd$rugged)
dd$cid <- ifelse( dd$cont_africa==1 , 1 , 2 )
dat_list <- list(
  log_gdp_std = dd$log_gdp_std,
  rugged_std = dd$rugged_std,
  cid = dd$cid
)
```

Then, we can define the model using ulam:

```r
m9.1 <- ulam(
  alist(
    log_gdp_std ~ dnorm(mu, sigma),
    mu <- a[cid] + b[cid] * rugged_std,
    a[cid] ~ dnorm(0, 0.1),
    b[cid] ~ dnorm(0, 0.3),
    sigma ~ dexp(1)
  ),
  data = dat_list,
  chains = 4,
  cores = 4
)
```

This model estimates the relationship be

In [47]:
question = "Give an example how to create a dashboard"

answer = ask_question(question)
print(answer)

I cannot answer this question based on the provided context. The provided chapters from "Chapter 3: The R Programming Language" discuss getting help in R, the R programming language, and its user interface, but do not provide information on creating a dashboard. 

This information is from Chapter 3: The R Programming Language by Krushcke J.K. However, the chapters provided do not contain sufficient information to create a dashboard.
